# classification with xgboost
model v1
- preliminary model
- data without dealing with imbalanced classification

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# data
import pandas as pd

# chosen models
from xgboost import XGBClassifier

# model training selection
from sklearn.model_selection import train_test_split

## model evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import cohen_kappa_score

In [2]:
df = pd.read_csv('../data/combined_feng.csv', index_col=0)
df.shape

(3000, 7353)

In [3]:
df.head(3)

,Number of Founders,Number of Funding Rounds,Trend Score (7 Days),Trend Score (30 Days),Trend Score (90 Days),Early Stage Venture,M&A,Seed,Made Acquisitions,Made Acquisitions; Was Acquired,...,last_funding_amount,cvr,last_funding_amount_usd,last_equity_funding_amount,last_equity_funding_amount_usd,total_equity_funding_amount,total_equity_funding_amount_usd,total_funding_amount,total_funding_amount_usd,female_led
Organization Name,,,,,,,,,,,,,,,,,,,,,
CMC,1,1,-0.4,-0.8,-1.2,1.0,0.0,0.0,0.0,0.0,...,10000000000,0.16,1.600000e+09,10000000000,1.600000e+09,10000000000,1.600000e+09,10000000000,1.600000e+09,0
Ping An Healthcare Management,1,1,-0.1,-0.2,-0.4,1.0,0.0,0.0,0.0,0.0,...,1150000000,1.00,1.150000e+09,1150000000,1.150000e+09,1150000000,1.150000e+09,1150000000,1.150000e+09,0
LeSee,1,1,-0.4,0.0,-0.5,1.0,0.0,0.0,0.0,0.0,...,1080000000,1.00,1.080000e+09,1080000000,1.080000e+09,1080000000,1.080000e+09,1080000000,1.080000e+09,0


### split data

In [4]:
# get data
X = df.drop(columns='female_led')
y = df['female_led']

In [5]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## model evaluation

In [6]:
def metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred) #perfect=1
    precision = precision_score(y_true, y_pred) #perfect=1
    recall = recall_score(y_true, y_pred) #perfect=1
    f1 = f1_score(y_true, y_pred) #perfect=1
#     roc_auc = roc_auc_score(y, clf.decision_function(X)) #perfect=1
#     log = log_loss(y_true, y_pred) #perfect=0
    mcc = matthews_corrcoef(y_true, y_pred) #perfect=1
    kappa = cohen_kappa_score(y_true, y_pred) #perfect=1
    
#     print(f'accuracy={accuracy},\
#             precision={precision}, recall={recall}, \
#             f1={f1}, mcc={mcc}, kappa={kappa}')

    return [accuracy, precision, recall, f1, mcc, kappa]

In [7]:
def evaluate(model, X_train=X_train, X_test=X_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return metrics(y_test, y_pred)

## machine learning

In [8]:
xgb = XGBClassifier()

In [9]:
metrics_xgb = evaluate(xgb)
metrics_xgb

[21:13:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy=0.9575757575757575,            precision=0.14285714285714285, recall=0.02702702702702703, f1=0.045454545454545456,            mcc=0.0469293226254877, kappa=0.033966823103015686


Result is high in accuracy but very low in precision or recall rate because the classification label is extremely imbalanced: the negative class takes up 95% of the dataset.

In [10]:
y.value_counts()

0    2858
1     142
Name: female_led, dtype: int64

In [11]:
2858/3000

0.9526666666666667